# Amazon Reviews
- creating initial reviews table from marketplace data set

- NOTE - In Marketplace Search for Amazon and select the Bright Data Amazon - Best Seller Products /Reviews/Products Dataset (Free for 1000 rows)

In [0]:
%sql
select * from bright_data_amazon_best_seller_products_reviews_products_dataset.datasets.amazon_best_seller_products limit 10;

In [0]:
%sql
select * from bright_data_amazon_best_seller_products_reviews_products_dataset.datasets.amazon_products LIMIT 10;

In [0]:
%sql
select * from bright_data_amazon_best_seller_products_reviews_products_dataset.datasets.amazon_reviews LIMIT 10;

In [0]:
%sql
select count(*) from bright_data_amazon_best_seller_products_reviews_products_dataset.datasets.amazon_reviews;

In [0]:
# Parameters for cleanup
catalog_name = "juan_dev"
schema_name = "genai"

In [0]:
DB_TBL = f"{catalog_name}.{schema_name}.amazon_reviews_mktpl"  # If you change the database name here, update all database names in other parts.

In [0]:
df = spark.table("bright_data_amazon_best_seller_products_reviews_products_dataset.datasets.amazon_reviews")

In [0]:
from pyspark.sql.functions import col

# Cast columns to double
df = df.withColumn("PRODUCT_RATING", col("PRODUCT_RATING").cast("double"))
df = df.withColumn("PRODUCT_RATING_MAX", col("PRODUCT_RATING_MAX").cast("double"))
df = df.withColumn("RATING", col("RATING").cast("double"))
df = df.withColumn("PRODUCT_RATING_COUNT", col("PRODUCT_RATING_COUNT").cast("double"))
df = df.withColumn("PRODUCT_REVIEWS_NUMBER", col("PRODUCT_REVIEWS_NUMBER").cast("double"))
df = df.withColumn("HELPFUL_COUNT", col("HELPFUL_COUNT").cast("double"))

# Write to Delta table with properties
# NOTE delta.enableChangeDataFeed to enable creation of vector search index.
df.writeTo(DB_TBL)\
    .using("delta")\
    .tableProperty("delta.minReaderVersion", "2")\
    .tableProperty("delta.minWriterVersion", "7")\
    .tableProperty("delta.enableChangeDataFeed", "true")\
    .createOrReplace()

In [0]:
from pyspark.sql.functions import rand

df = spark.table(DB_TBL)